# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление

1. [Описание данных](#1)
* [Импорт модулей](#2) 
* [Функции](#3)
* [Загрузка данных](#4)
* [Предобработка данных](#5)
    * [Обработка пропусков](#)
    * [Замена типа данных](#)
    * [Обработка дубликатов](#)
    * [Лемматизация](#)
    * [Категоризация](#)
* [](#)
* [](#)
* [Общий вывод](#conclusion)

<a id='1'></a>
## 1. Описание данных

* *children* — количество детей в семье
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита

<a id='2'></a>
## 2. Импорт модулей

In [1]:
!pip install pymystem3 pymorphy2 pymorphy2-dicts-ru

You should consider upgrading via the '/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pymorphy2                      # для определения частей речи
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
from nltk.stem import SnowballStemmer # для стемминга

<a id='3'></a>
## Функции

In [3]:
def clean_lemmas_rating(rating_dict):
    '''
    Функция для удаления предлогов, пробелов, символа конца строки.
    На вход принимаем список кортежей (<count>, <word>):
    count - частота встречаемости слова во тестовой выборке
    word - слово из целей кредита
    '''
    
    result = []
    
    for item in rating_dict.items():
        count, word = item
        ps = morph.parse(word)[0].tag.POS
        
        try:
            # часть речи (part of speech)
            ps = morph.parse(word)[0].tag.POS
        except:
            # проблемы с кодировкой или внезапно другой язык в данных
            # оставляем результат без изменений, не смогли обработать
            print('Error - processing: {} ->'.format(word), rating)
            
        # оставляем только существительные
        if ps == 'NOUN':
            result.append((word, count))
            
    return dict(result)

In [4]:
def purpose_category(purpose, rating):
    '''
    Функция для определения категорий целей кредита.
    Проходим от наиболее популярного слова в rating до наименее популярного.
    При первом совпадениии возвращаем категорию.
    rating - аргумент по умолчанию
    '''
    
    for category in rating:
        # получаем основу слова категории, так как в целях используются разные падежи
        stemmed_category = russian_stemmer.stem(category)
        
        if stemmed_category in purpose:
            return category

    return 'NaN'

In [5]:
def get_debt_probability(df, idx, col='gender', val='debt'):
    """
    Функция для создания совдных таблиц по разным параметрам
    """

    df_debt = person_data.groupby(idx)[val].sum().to_frame().reset_index()
    df_all = person_data.groupby(idx)[col].count().to_frame().reset_index()
    df_data = df_debt.merge(df_all,
                            on=idx,
                            how='inner')
    df_data.rename(columns={col:"all"}, inplace=True)

    # создаем столбец с вероятностью долга
    probability = val+'_probability'
    df_data[probability] = (df_data[val] / df_data['all'] * 100).round(1)
    df_data.sort_values(probability, ascending=False, inplace=True)
    
    return df_data

<a id='4'></a>
## Загрузка данных

In [6]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
# переименуем столбцы
data.rename(columns={"dob_years": "age"}, inplace=True)

data.sample(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14568,0,-3423.447872,47,среднее,1,женат / замужем,0,M,компаньон,0,102151.269558,недвижимость
11787,0,-772.740198,45,среднее,1,гражданский брак,1,M,сотрудник,0,402526.010628,операции с недвижимостью
4836,0,-842.077429,63,среднее,1,женат / замужем,0,M,компаньон,0,68159.059436,свой автомобиль
18036,1,-2666.711710,28,среднее,1,в разводе,3,F,сотрудник,0,82051.948696,дополнительное образование
8342,1,-1862.044118,28,высшее,0,женат / замужем,0,F,госслужащий,0,89489.878446,профильное образование
10703,0,-3619.862152,35,среднее,1,женат / замужем,0,F,сотрудник,0,89817.943638,получение образования
9115,0,-5729.555296,56,среднее,1,женат / замужем,0,F,сотрудник,0,97751.771461,жилье
20414,1,-4982.874406,58,среднее,1,женат / замужем,0,F,компаньон,0,78037.175217,операции со своей недвижимостью
14656,0,-882.068337,27,среднее,1,женат / замужем,0,M,сотрудник,0,87279.982150,заняться образованием
2140,0,-11121.098517,53,среднее,1,гражданский брак,1,F,компаньон,0,79344.761601,покупка коммерческой недвижимости


In [8]:
null_days_employed = data[ data['days_employed'].isnull()].shape[0]
null_total_income = data[ data['total_income'].isnull()].shape[0]

print(null_days_employed, null_total_income)

2174 2174


In [9]:
data_no_days_employed = data[data['days_employed'].isnull()]
data_no_days_employed['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Данные представляют включают признаки 21525 объектов. Выявлены следующие особенности:
1. Пропуски в столбцах _days_employed,  total_income_ в одинаковом количестве - 2174.
2. Разнородность описания:
   * столбец _gender_ неинформативно описывает пол заемщика;
   * встречаются одинаковые значения, написанные в разном регистре.
3. Отрицательные значения столбца _days_employed_ .
4. Типы данных столбцов: числовые (целочисленные и дробные), строковые.
4. Столбцы _family_status, purpose_ - категориальные признаки.
5. Столбец _purpose_ содержит значения, часто отличающиеся падежами (с включениями предлогов), но имеющие одинаковый смысл: "на покупку автомобиля", "на покупку своего автомобиля", "приобретение автомобиля".

<a id='5'></a>
## Предобработка данных

<a id='gaps'></a>
### Обработка пропусков

In [10]:
# взаимосвязь пропусков в 'days_employed' и 'total_income' с значением 'income_type'
null_data = data[data.days_employed.isnull() | data.total_income.isnull()][
    ['days_employed','income_type', 'total_income']]

if null_days_employed == null_data.shape[0] and null_total_income == null_data.shape[0]:
    print('Пропуски для total_income и days_employed совпадают:', null_data.shape[0])

Пропуски для total_income и days_employed совпадают: 2174


In [11]:
# Количество пропусков 'total_income' для каждого типа занятости
data[data['total_income'].isnull()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [12]:
# Количество пропусков 'days_employed' для каждого типа занятости
data[data['days_employed'].isnull()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [13]:
# Посмотрим на значения возраста
data.groupby('age')['education'].count().head()

age
0     101
19     14
20     51
21    111
22    183
Name: education, dtype: int64

Во многих заявках возраст не указан, поэтому посмотрим, для каких типов занятости встречается нулевой возраст.

In [14]:
data[data['age'] == 0]['income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

Для столбцов **income_type, days_employed, age** встречаются пропуски. Составим отдельную таблицу со средними и медианными значениями по каждому проблемному столбцу.

In [15]:
# считаем для age
income_type_stat = data.groupby('income_type').agg({'age':['median','mean']})

# считаем для days_employed
income_type_stat = income_type_stat.merge(data.groupby('income_type').agg({'days_employed':['median','mean']}),
                                          on='income_type',
                                          how='inner')

# считаем для total_income
income_type_stat = income_type_stat.merge(data.groupby('income_type').agg({'total_income':['median','mean']}),
                                          on='income_type',
                                          how='inner')

income_type_stat

age             days_employed                 \
                median       mean         median           mean   
income_type                                                       
безработный       38.0  38.000000  366413.652744  366413.652744   
в декрете         39.0  39.000000   -3296.759962   -3296.759962   
госслужащий       40.0  40.636737   -2689.368353   -3399.896902   
компаньон         39.0  39.697542   -1547.382223   -2111.524398   
пенсионер         60.0  59.063019  365213.306266  365003.491245   
предприниматель   42.5  42.500000    -520.848083    -520.848083   
сотрудник         39.0  39.821027   -1574.202821   -2326.499216   
студент           22.0  22.000000    -578.751554    -578.751554   

                  total_income                 
                        median           mean  
income_type                                    
безработный      131339.751676  131339.751676  
в декрете         53829.130729   53829.130729  
госслужащий      150447.935283  170898.309923  
компаньон        172357.950966  202417.461462  
пенсионер        118514.486412  137127.465690  
предприниматель  499163.144947  499163.144947  
сотрудник        142594.396847  161380.260488  
студент           98201.625314   98201.625314

Заполним пропуски для столбцов **age, days_employed, total_income** медианой по отдельным типам занятости, так как среднее арифметическое расходится с медианным значением.

In [16]:
# по столбцам, где будем заполнять пропуски
for column in set([ i[0] for i in income_type_stat.columns ]):
    # по типам занятости
    for group in income_type_stat.index.to_list():
        fill_value = income_type_stat[column]['median'].loc[group]
        
        if column == 'age':
            data[data['income_type'] == group] = data[data['income_type'] == group].replace({column: {0: fill_value}})
# 
        else:
            data[data['income_type'] == group] = data[data['income_type'] == group].fillna(fill_value)


print('Минимальный возраст в данных:', data['age'].value_counts().sort_index().head(1).idxmax())
print('Количество пропущенных значений days_employed:', data[data['days_employed'].isnull()].shape[0])
print('Количество пропущенных значений total_income:', data[data['total_income'].isnull()].shape[0])


Минимальный возраст в данных: 19.0
Количество пропущенных значений days_employed: 0
Количество пропущенных значений total_income: 0


<div style="border:solid gold 2px; padding: 20px"> <h1 style="color:gold; margin-bottom:20px">Комментарий наставника</h1>

Здесь не стоило делать через for - это медленно, лучше было сделать вот так 
    
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

In [17]:
# посмотрим на значения данных о поле заемщиков
data['gender'] = data['gender'].replace('M', 'мужской')
data['gender'] = data['gender'].replace('F', 'женский')
print(data['gender'].value_counts())

женский    14236
мужской     7288
XNA            1
Name: gender, dtype: int64


In [18]:
popular_gender = data['gender'].value_counts().idxmax()
print('Самый популярный пол:', popular_gender)

Самый популярный пол: женский


Заменим пропуск на значение самого популярного пола заемщиков.

In [19]:
data.loc[data['gender'] == 'XNA', 'gender'] = popular_gender
print("\n\nДанные 'gender' после обработки:\n")

# print(data['gender'].value_counts())
data.info()



Данные 'gender' после обработки:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   age               21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


Проверка пропущенных данных о днях занятости __days_employed__ и доходе заемщика __total_income__ показала, что пропуски характерны для одних и тех же объектов. Однако, это не значит, что лица, у которых пропущены значения в этих стобцах, не работают на момент заявки: данные по ошибке могли быть не заполнены или потеряться. Также было обнаружено, что в данных о возрасте в столбце __age__ много нулевых значений (101), что может повлиять на результат в дальнейшем анализе. Поэтому было принято решение заменить пропуски на медианные значения в зависимости от типа занятости.
Также обнаружен неявный (типа Str) единичный пропуск в данных о поле заемщиков и заменен на самый часто втречаемый пол в данных - _женский_.

<a id='types'></a>
### Замена типа данных

In [20]:
# преобразование типа данных в столбцах типа float64

for column in ['days_employed', 'age', 'total_income']:
    data[column] = data[column].astype('int')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   age               21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Столбцы __days_employed__, __age__, __total_income__ в выборке были типа _float64_ без округления. Во избежание проблем при поиске дубликатов данных, значения этих столбцов были переведены в целочисленный тип данных.

<a id='duplicates'></a>
### Обработка дубликатов

In [21]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [22]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [23]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [24]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Переведем значения категориальных перевенных к нижнему регистру.

In [25]:
for column in ['education', 'family_status']:
    data[column] = data[column].str.lower()

for column in [ 'education', 'family_status', 'gender', 'income_type', 'purpose']:
    print(data[column].value_counts().index.to_list())

['среднее', 'высшее', 'неоконченное высшее', 'начальное', 'ученая степень']
['женат / замужем', 'гражданский брак', 'не женат / не замужем', 'в разводе', 'вдовец / вдова']
['женский', 'мужской']
['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель', 'безработный', 'в декрете', 'студент']
['свадьба', 'на проведение свадьбы', 'сыграть свадьбу', 'операции с недвижимостью', 'покупка коммерческой недвижимости', 'операции с жильем', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью', 'покупка жилья', 'жилье', 'покупка жилья для семьи', 'строительство собственной недвижимости', 'недвижимость', 'операции со своей недвижимостью', 'строительство жилой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'строительство недвижимости', 'ремонт жилью', 'покупка жилой недвижимости', 'на покупку своего автомобиля', 'заняться высшим образованием', 'автомобиль', 'сделка с подержанным автомобилем', 'свой автомобиль', 'на покупку подержанного автомобиля', 'авто

In [26]:
# сколько дубликатов
data[data.duplicated()].count()

children            71
days_employed       71
age                 71
education           71
education_id        71
family_status       71
family_status_id    71
gender              71
income_type         71
debt                71
total_income        71
purpose             71
dtype: int64

In [27]:
# Удаляем полностью совпадающие строки и упорядочиваем индексы

data = data.drop_duplicates().reset_index(drop=True)
data[data.duplicated()].count().sum()

0

В исходной таблице обнаружено 71 полностью идентичных строк, которые могли появиться в силу разных причин:
* несколько заявлений от одного человека
* ошибка оператора
* техническая неисправность

Все дубликаты удалены.

<a id='lemmas'></a>
### Лемматизация

In [28]:
m = Mystem()

In [29]:
# объединяем все цели покупки в один текст для удобства лемматизации
purpose_text = ' '.join(data['purpose'].to_list())

# находим леммы
purpose_lemmas = m.lemmatize(purpose_text)

# считаем количество каждой
lemmas_dict = dict(Counter(purpose_lemmas))

# удаляем пробелы, символ конца строки, некоторые предлоги
for sym in [' ', 'с', 'на', 'для', 'со', '\n']:
    del lemmas_dict[sym]

# и сортируем по убыванию
lemmas_dict_sort = dict(sorted([(value, key) for (key,value) in lemmas_dict.items()], reverse=True))

# 10 самых популярных лемм
lemmas_dict_sort

{6351: 'недвижимость',
 5897: 'покупка',
 4460: 'жилье',
 4306: 'автомобиль',
 4013: 'образование',
 2604: 'операция',
 2324: 'свадьба',
 2230: 'свой',
 1878: 'строительство',
 1374: 'высокий',
 1314: 'получение',
 1311: 'коммерческий',
 1230: 'жилой',
 941: 'сделка',
 906: 'дополнительный',
 904: 'заниматься',
 853: 'подержать',
 768: 'проведение',
 765: 'сыграть',
 651: 'сдача',
 638: 'семья',
 635: 'собственный',
 607: 'ремонт',
 461: 'приобретение',
 436: 'профильный',
 111: 'подержанный'}

В десятке популярных такие цели кредита, как _недвижимость, жилье, автомобиль, образование, свадьба, строительство_.

<a id='categories'></a>
### Категоризация данных

In [30]:
# добавление стажа работы в годах
data['years_employed'] = (data['days_employed']/365).astype('int')

In [31]:
# разделим данные на 3 таблицы

# 1 - person_data - основные данные
person_data = data[['income_type', 'years_employed', 'age', 'total_income','purpose', \
                    'gender', 'debt', 'children', 'family_status_id', 'education_id']]

# 2 - family_dict - соответствие 'тип семейного положения' - 'идентификатор'
education_dict = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

# 3 - education_dict - соответствие 'тип образования' - 'идентификатор'
family_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

# данные таблицы person_data
person_data.head(5)

,income_type,years_employed,age,total_income,purpose,gender,debt,children,family_status_id,education_id
0,сотрудник,-23,42,253875,покупка жилья,женский,0,1,0,0
1,сотрудник,-11,36,112080,приобретение автомобиля,женский,0,1,0,1
2,сотрудник,-15,33,145885,покупка жилья,мужской,0,0,0,1
3,сотрудник,-11,32,267628,дополнительное образование,мужской,0,3,0,1
4,пенсионер,932,53,158616,сыграть свадьбу,женский,0,0,1,1


In [32]:
# данные таблицы education_dict
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [33]:
# данные таблицы family_dict
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [34]:
# Посмотрим значения квантилей дохода
# val_25, val_50, val_75 = tuple(
#     person_data['total_income'].describe().loc[['25%','50%', '75%']].values)
# print(val_25, val_50, val_75)

In [35]:
# доп. проверка
data2, bins = pd.qcut(person_data['total_income'], 4, retbins=True)
bins

array([  -2689.  ,   92839.  ,  138975.  ,  201340.25, 2265604.  ])

In [36]:
def income_category(income, val_25, val_50, val_75):
    """
    Функция для разбиения на категории уровню дохода
    income - значения дохода
    """
    
    if income < val_25:
        return 'низкий'
    
    if val_25 <= income <= val_50:
        return 'ниже среднего'
    
    if val_50 < income <= val_75:
        return 'средний'
    
    else:
        return 'высокий'

In [37]:
# значения квантилей
q_25, q_50, q_75 = tuple(
    person_data['total_income'].describe()[['25%','50%', '75%']].to_list())
    
# добавление нового столбца - категория заемщика по уровню дохода
person_data['income_category'] = (
    person_data['total_income']
    .apply(income_category, args=(q_25, q_50, q_75))
)

person_data['income_category'].value_counts()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


высокий          5364
низкий           5364
средний          5363
ниже среднего    5363
Name: income_category, dtype: int64

In [38]:
morph = pymorphy2.MorphAnalyzer()

lemmas_rating = clean_lemmas_rating(lemmas_dict_sort)
lemmas_rating

{'недвижимость': 6351,
 'покупка': 5897,
 'жилье': 4460,
 'автомобиль': 4306,
 'образование': 4013,
 'операция': 2604,
 'свадьба': 2324,
 'строительство': 1878,
 'получение': 1314,
 'жилой': 1230,
 'сделка': 941,
 'проведение': 768,
 'сдача': 651,
 'семья': 638,
 'ремонт': 607,
 'приобретение': 461}

Удалим из рейтинга слова, означающие действие, а не цель.

In [39]:
rm_words = ['покупка', 'получение', 'сделка', 'проведение', 
            'приобретение', 'сдача', 'семья', 'жилой']
for w in rm_words:
    del lemmas_rating[w]
    
lemmas_rating

{'недвижимость': 6351,
 'жилье': 4460,
 'автомобиль': 4306,
 'образование': 4013,
 'операция': 2604,
 'свадьба': 2324,
 'строительство': 1878,
 'ремонт': 607}

In [42]:
russian_stemmer = SnowballStemmer('russian')

# добавление категории заемщика по цели кредита
person_data['purpose_category'] = person_data['purpose'].apply(
    purpose_category, args=(lemmas_rating,))

# заменим 'жилье' -> 'недвижимость'
person_data['purpose_category'] = person_data['purpose_category'].replace('жилье', 'недвижимость')

# посмотрим количество клиентов для каждой категории - если нет NaN, то 
# все прошло успешно
person_data['purpose_category'].value_counts()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

Данные были разделены на 3 таблицы:
1. __person_data__    - основные данные заемщика
* __family_dict__    - таблица-словарь для соответствия _'тип семейного положения' - 'идентификатор'_
* __education_dict__ - таблица-словарь _'тип образования' - 'идентификатор'_

Согласно поставленной задаче было решено разбить данные на категории по следующим признакам:

* категории __количества детей__

    В зависимости от количества детей заемщик может быть в категории _"бездетные", "с детьми" (1 или 2), "многодетные"_. Заемщики с отрицательным значением в данных о количестве детей считаем бездетными (либо ошибка оператора/системы, либо таким образом указываются совершеннолетние дети)
    
    
* категории __семейного положения__

    В зависимости от семейного положения при заёме учитывается совместный доход супругов. Соответственно, добавлены две категории _"в браке"_ и _"свободные"_.


* категории __уровня дохода__

    Доход в данных был разделен на _"низкий" (< 30000 у.е.), "средний" (30000 - 799999 у.е.), "высокий" (>= 80000 у.е.)_. Пороговые значения выбраны приблизительно, так как неизвестно, какому географическому и экономическому объекту соответствует тестовая выборка.




* категории __целей кредита__

    Процесс определения категорий определялся исходя из результата лемматизации - был получен рейтинг самых часто встречаемых слов в целях, затем проводилась автоматизированная очистка данных фнукцией clean_lemmas_rating() от непрезентабельных для дальнейшего анализа частей речи - всех, кроме существительных. Дополнительно из рейтинга были удалены те существительные, которые не несут смысловой нагрузки категоризации. В результате работы функции purpose_category(), примененной к таблице, данные были разбиты на 4 категории
    
    1. Недвижимость
    * Автомобиль
    * Образование
    * Свадьба
    
    Возможная неточность в определении категорий - слово _недвижимость_ употребляется в данных в качестве _жилой_ и _коммерческой_ (не жилой).

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
# так как группы с количеством заемщиков <100 составляют малую часть от других групп (где счет идет на сотни и
# тысячи) и в процентном соотношении дают вбросы, не будем учитывать эти группы при ответе на поставленный вопрос

# считаем вероятность задолжности (debt_probability)
debt_children_data = get_debt_probability(person_data, 'children', col='gender', val='debt')
debt_children_data[debt_children_data["all"] > 100][['children', 'debt_probability']]

,children,debt_probability
3,2,9.5
2,1,9.2
4,3,8.2
1,0,7.5


### Вывод

Выделен показатель в виде вероятности задолженности:
- у бездетнных людей показатель меньше всего - 7,5%;
- люди с 3 детьми вероятность выше,но не намного - 8,2%;
- заемщики с 1-2 детьми характеризуются близкой вероятностью задолжности - 9,2% и 9,5% соответственно.

При сравнении групп "бездетные", "с одним ребенком" и "с двумя детьми", где группы сравнимы по количеству человек, видно зависимость, что при увеличении количества детей вероятность задолжности увеличивается (нелинейно).

Однако, при рассмотрении дополнительно группы "с 3 детьми" заметно отклонение от сделанного вывода - вероятность снижается с 9,5% до 8,2%. В данном случае группа значительно отличается по численности от остальных - 330 человек против тысячных групп. Соответственно, для более основательного вывода нужно больше данных о людях с количеством детей 3 и более.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
# зависимость долгов от семейного положения
debt_family_data = get_debt_probability(person_data, 'family_status_id', col='gender', val='debt')

# добавим расшифровку статуса семейного положения
debt_family_data = debt_family_data.merge(family_dict,
                                         on='family_status_id',
                                         how='inner')
debt_family_data[['family_status', 'debt_probability']]

,family_status,debt_probability
0,не женат / не замужем,9.8
1,гражданский брак,9.3
2,женат / замужем,7.5
3,в разводе,7.1
4,вдовец / вдова,6.6


### Вывод

Выявлена зависимость между семейным положением и возвратом кредита в срок:
* Наименьшая вероятность задолжности у вдовцов/вдов (самая немногочисленная группа) - 6.6%.
* Для людей в разводе и браке вероятность больше - 7.1% и 7.5% соотвественно.
* В гражданском браке вероятность возрастает - 9.3%.
* 9.8% долгов соотвествует категории людей, которые не состоят в отношениях.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
# # создаем сводную таблицу, чтобы увидеть зависимость долгов от уровня доходов
# income_debt_pivot = create_pivot_table(person_data, 'income_category')
# income_debt_pivot

# считаем вероятность задолжности (debt_probability)
debt_income_data = get_debt_probability(person_data, 'income_category', col='gender', val='debt')
debt_income_data[['income_category', 'debt_probability']]

,income_category,debt_probability
3,средний,9.0
1,ниже среднего,8.4
2,низкий,7.9
0,высокий,7.2


### Вывод

Выявлена зависимость между уровнем дохода и возвратом кредита в срок:
* наименьшая вероятность задолжности у категорий людей с высоким и низким уровнем дохода - 7.1% и 8%;
* большая вероятность долга у людей со средним уровнем дохода и ниже среднего - 8.5% и 8.8%.

- Как разные цели кредита влияют на его возврат в срок?

In [46]:
# считаем вероятность задолжности (debt_probability)
debt_purpose_data = get_debt_probability(person_data, 'purpose_category', col='gender', val='debt')
debt_purpose_data[['purpose_category', 'debt_probability']]

,purpose_category,debt_probability
0,автомобиль,9.4
2,образование,9.2
3,свадьба,8.0
1,недвижимость,7.2


### Вывод

Выявлена зависимость между целями кредита и возвратом кредита в срок. Согласно полученным результатам, выявлены следующие вероятности для категорий целей кредита:
* "автомобиль" и "образованиее" имеют наибольший показатель - 9.4% и 9.2% соответственно;
* "свадьба" - 8%;
* "недвижимость" - категория с наименьшей вероятностью  долга - 7.2%.

<div style="border:solid red 2px; padding: 20px"> <h1 style="color: red; margin-bottom:20px">Комментарий наставника</h1>

Ответы на вопросы сделаны не верно. Для каждой группы надо посчитать вероятность невозврата кредита, т.е. количество должников / размер группы. Т.е если у нас 100 человек без детей и  из них 10 с долгом, то вероятность невозврата 10%. И так для каждого вопроса и для каждой группы.
    
Да, ты верно использовала здесь pivot_table, но группировка должна быть другой

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошо, теперь все верно.

<a id='conclusion'></a>
### Общий вывод

Данные о заемщиках представлены несколькими признаками, представляющие собой коллчественные и категориальные значения. Значение целей кредита представлены разнородными категориальными переменными, так как при одинаковых смысловых значениях указаны в разных падежах.

При исследовании в данных были обнаружены проблемы:

* __Неинформативные названия столбцов__. Столбец __dob_years__ был переименован на __age__.


* __Пропуски__. В данных найдены явные пропуски, которые могут свидетельствовать о технической проблеме выгрузки или сбора данных. Пропущенные значения о _стаже, общем доходе, возрасте_ заменены на медианные значения в зависимости от типа занятости, а единичный пропуск о _поле_ заемщика был заменен на самый часто встречаемый - _женский_.


* __Разнородность описания данных__. В данных пол заемщиков указан неинформативно - одной латинской буквой. Значения были заменены на значения _"женский"_ и _"мужской"_. Также в данных были указаны идентичные значения в разном регистре. Как результат, все значения приведены к нижнему регистру.


* __Некорректные значения__. В столбцах о стаже и доходе встречаются отрицательные значения. Помимо этого, у некоторых заемщиков указан завышенный стаж работы - более 100 лет. Так как данные о стаже в поставленной задаче не будут учитываться при анализе, значания оставлены без изменений. В доходе отрицательные значения не в большом количестве относительно количества данных, поэтому отрицательные значения дохода также оставлены.


* __Типы данных__. Значения с типом с плавающей запятой (указанные без округления) переведены в целочисленный во избежание ошибок при сравнении количественных переменных и поиске дубликатов.


* __Дубликаты__. В исходной таблице обнаружена совокупность полностью идентичных строк, которые могли появиться в следствие технической проблемы или наличии повторных заемщиков. Так как не поставлена задача исследования повторных заемщиков, полностью совпадающие строки были удалены.


В результате лемматизации были выделены самые популярные цели кредита:
* недвижимость
* жилье
* автомобиль
* образование
* свадьба
* строительство


Для упрощения работы с данными выполнено разделение на 3 таблицы:
1. __person_data__    - основные данные заемщика
* __family_dict__    - таблица-словарь для соответствия _'тип семейного положения' - 'идентификатор'_
* __education_dict__ - таблица-словарь _'тип образования' - 'идентификатор'_


Согласно поставленной задаче было решено разбить данные на категории по следующим признакам:

* категории __уровня дохода__ (низкий, средний, высокий) - пороговые значения выбраны приблизительно, так как
    неизвестно, какому географическому и экономическому объекту соответствуют данные.
* категории __целей кредита__ (недвижимость, жилье, автомобиль, образование, свадьба)


Выявлены следующие зависимости в данных:

* между наличием детей и возвратом кредита в срок - "бездетные", "с одним ребенком" и "с двумя детьми", где группы сравнимы по количеству человек, видно зависимость, что при увеличении количества детей вероятность задолжности нелинейно увеличивается - __7.5%__, __9.2%__, __9.5__% соответственно. Для основательного вывода о зависимости нужно больше данных о людях с количеством детей 3 и более.

* между семейным положением и возвратом кредита в срок. Наименьшая вероятность задолжности у вдовцов/вдов (самая немногочисленная группа) - 6.6%, для людей в разводе и браке вероятность больше - __7.1__% и __7.5__%, а для людей в гражданском браке и свободных вероятность возрастает - __9.3%__ и  __9.8%__.
    
* между уровнем дохода и возвратом кредита в срок. Наименьшая вероятность задолжности у категорий людей с высоким и низким уровнем дохода - __7.1%__ и __8%__. Большая вероятность долга у людей со средним уровнем дохода и ниже среднего - __8.5%__ и __8.8%__.

* между целями кредита и возвратом кредита в срок. Категории "автомобиль" и "образованиее" имеют наибольший показатель - __9.4%__ и __9.2%__, "свадьба" - __8%__, "недвижимость" - категория с наименьшей вероятностью  долга - __7.2%__.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.